In [ ]:
objname = "yasone2"

In [ ]:
import astroquery
import astropy.units as u
from astropy import coordinates

In [ ]:
radec = {
    "yasone2": [262.348711541485, 6.38646541032259]
}[objname]

In [ ]:
coord0 = coordinates.SkyCoord(radec[0], radec[1], unit=u.deg)

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
Gaia.ROW_LIMIT = 100_000
gaia_sources = Gaia.query_object(coordinate=coord0, radius=30*u.arcmin, verbose=True)

In [ ]:
len(gaia_sources)

In [ ]:
assert len(gaia_sources) < Gaia.ROW_LIMIT

In [ ]:
gaia_sources

In [ ]:
gaia_sources["solution_id"].dtype

In [ ]:
import numpy as np

In [ ]:
gaia_sources

In [ ]:
good_cols = [gaia_sources[col].dtype != np.dtypes.ObjectDType for col in gaia_sources.columns]

In [ ]:
good_cols

In [ ]:
bad_cols = np.array(gaia_sources.colnames)[~np.array(good_cols)]
bad_cols

In [ ]:
gaia_sources_clean = gaia_sources[*np.array(gaia_sources.colnames)[good_cols]]

In [ ]:
save_table_as_ldac(gaia_sources_clean, f"{objname}_Gaia.cat")

In [ ]:
def save_table_as_ldac(tbl, filename, **kwargs):
    """
    Save a table as a fits LDAC file
    
    Parameters
    ----------
    tbl: `astropy.table.Table`
        Table to save
    filename: str
        Filename to save table
    kwargs:
        Keyword arguments to pass to hdulist.writeto
    """
    hdulist = convert_table_to_ldac(tbl)
    hdulist.writeto(filename, **kwargs)

In [ ]:
convert_table_to_ldac(gaia_sources_clean)

In [ ]:
gaia_sources_clean.write(f"{objname}_Gaia.fits")

In [ ]:
def convert_table_to_ldac(tbl):
    """
    Convert an astropy table to a fits_ldac
    
    Parameters
    ----------
    tbl: `astropy.table.Table`
        Table to convert to ldac format
    Returns
    -------
    hdulist: `astropy.io.fits.HDUList`
        FITS_LDAC hdulist that can be read by astromatic software
    """
    from astropy.io import fits
    import tempfile
    f = tempfile.NamedTemporaryFile(suffix='.fits', mode='rb+')
    tbl.write(f, format='fits')
    f.seek(0)
    hdulist = fits.open(f, mode='update')
    tbl1, tbl2 = convert_hdu_to_ldac(hdulist[1])
    new_hdulist = [hdulist[0], tbl1, tbl2]
    new_hdulist = fits.HDUList(new_hdulist)
    return new_hdulist

In [ ]:
# Copyright 2015 Fred Moolekamp
# BSD 3-clause license
"""
Functions to convert FITS files or astropy Tables to FITS_LDAC files and
vice versa.
"""
def convert_hdu_to_ldac(hdu):
    """
    Convert an hdu table to a fits_ldac table (format used by astromatic suite)
    
    Parameters
    ----------
    hdu: `astropy.io.fits.BinTableHDU` or `astropy.io.fits.TableHDU`
        HDUList to convert to fits_ldac HDUList
    
    Returns
    -------
    tbl1: `astropy.io.fits.BinTableHDU`
        Header info for fits table (LDAC_IMHEAD)
    tbl2: `astropy.io.fits.BinTableHDU`
        Data table (LDAC_OBJECTS)
    """
    from astropy.io import fits
    import numpy as np
    tblhdr = np.array([hdu.header.tostring(',')])
    col1 = fits.Column(name='Field Header Card', array=tblhdr, format='13200A')
    cols = fits.ColDefs([col1])
    tbl1 = fits.BinTableHDU.from_columns(cols)
    tbl1.header['TDIM1'] = '(80, {0})'.format(len(hdu.header))
    tbl1.header['EXTNAME'] = 'LDAC_IMHEAD'
    tbl2 = fits.BinTableHDU(hdu.data)
    tbl2.header['EXTNAME'] = 'LDAC_OBJECTS'
    return (tbl1, tbl2)